# 🧠 Spiking Neural Network Decoder
This notebook simulates neuron spike activity using the **Leaky Integrate-and-Fire (LIF)** model and decodes behavior from neural spike rates using machine learning.

---
## 📌 Objectives
- Simulate biologically realistic spikes with the LIF model
- Estimate firing rates from spike trains
- Classify behavior using firing rate patterns and ML models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

## 🔬 Step 1: Simulate Spikes using Leaky Integrate-and-Fire Model
The **Leaky Integrate-and-Fire (LIF)** model simulates a neuron's membrane potential over time. When the potential exceeds a threshold, a spike is triggered and the potential resets. An adaptive threshold models biological behavior more closely.

In [ ]:

def LeakyIF_3(input_currents=None, duration=0.2, dt=0.0001):
    tau = 0.020
    R = 3e7
    U_rest = -0.07
    theta = -0.030
    spikeVolt = 0.1
    backgroundI = 3e-9
    tau_adapt = 0.3
    increase_threshold = 0.012

    if input_currents is None:
        input_currents = {}
        U_0 = 0.3
        I_0 = U_0 / R
        psc = np.arange(0, duration, dt)
    else:
        U_0 = input_currents.get('U_0', 0.3)
        I_0 = U_0 / R
        psc = input_currents.get('psc', np.arange(0, duration, dt))

    ipsc = input_currents.get('ipsc', [])
    n_pcs = len(psc)
    U = np.zeros(n_pcs)
    spikes = np.zeros(n_pcs)
    threshold = np.zeros(n_pcs)
    U[0] = U_rest
    threshold[0] = theta

    for i in range(1, n_pcs):
        I_t = backgroundI + (I_0 if i in ipsc else 0)
        dU = (-(U[i-1] - U_rest) + R * I_t) / tau * dt
        U[i] = U[i-1] + dU
        threshold[i] = threshold[i-1] + ((theta - threshold[i-1]) / tau_adapt) * dt
        if U[i] >= threshold[i]:
            spikes[i] = 1
            U[i] = U_rest
            threshold[i] += increase_threshold
    return psc, U, spikes, threshold

# Generate and Plot
psc, U, spikes, threshold = LeakyIF_3()
plt.figure(figsize=(10, 4))
plt.plot(psc, U, label='Membrane Potential')
plt.plot(psc, threshold, 'r--', label='Threshold')
plt.title("Simulated Spike Train (LIF Neuron)")
plt.xlabel("Time (s)")
plt.ylabel("Potential (V)")
plt.legend()
plt.show()


## 📊 Step 2: Load and Prepare Dataset
We simulate a dataset mimicking real-world neuron firing rates. Each row is one trial and each column is a neuron's firing rate. The label represents the behavior performed.

In [ ]:

np.random.seed(42)
X = np.random.rand(120, 6)  # 6 neurons
y = np.random.choice([1, 2, 3, 4], size=120)

df = pd.DataFrame(X, columns=[f'Neuron_{i+1}' for i in range(6)])
df['Behavior'] = y
df.head()


## 🤖 Step 3: Train Machine Learning Classifiers
We use multiple models to predict behavior based on neuron activity. Models include:
- Support Vector Machine (SVM)
- Random Forest
- Linear Discriminant Analysis (LDA)
- AdaBoost

We measure each model's accuracy on a held-out test set.

In [ ]:

X = df.drop('Behavior', axis=1)
y = df['Behavior']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'LDA': LinearDiscriminantAnalysis(),
    'AdaBoost': AdaBoostClassifier()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f'{name} Accuracy: {acc:.2f}')


## 📉 Step 4: Evaluate Model Performance
A **confusion matrix** shows how well the classifier performed across different behavior categories. Ideally, predictions match true labels perfectly (diagonal matrix).

In [ ]:

best_model = RandomForestClassifier().fit(X_train, y_train)
y_pred = best_model.predict(X_test)

cm = confusion_matrix(y_test, y_pred, labels=[1, 2, 3, 4])
sns.heatmap(pd.DataFrame(cm, index=[1,2,3,4], columns=[1,2,3,4]), annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix - Random Forest")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()
